<a href="https://colab.research.google.com/github/gzholtkevych/Programming-Theory/blob/master/RecFunction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<H1><b>РЕКУРСТВНІ ФУНКЦІЇ</b></H1>

# Налаштування

In [2]:
from typing import Dict, Any, Tuple
from typing_extensions import Self

from functools import reduce

# Натуральні числа

## Аксіоматика Пеано

Множина натуральних чисел - це найменша множина, що містить $0$ та з кожним свохм елементом $n$ містить наступний елемент $S(n)$, причому

1. для будь-якого елемента $n$ цієї множини вірно $S(n)\neq0$;
2. з рівності $S(n)=S(m)$ для елементів цієї множини $n$ та $m$ випливає рівність $n=m$;
3. для будь-якого предиката $P$, визначеного на цій множині, вірність
    - $P(0)$ та
    - $P(m)\to P(S(m))$

    забезпечує вірність для $m$ з цієї множини забезпечує вірність $P(n)$ для всіх $n$ з цієї множини.

## Програмна реалізація

Пропонується реалізувати концепцію натурального числа як клас `nat`, який успадковується від вбудованого типу `int`.

Натуральним числом в цій реалізації є невід'ємне ціле число.

Для униможливлення створення двох натуральних чисел з однаковими значеннями цей клас містить статичний словник `__used`, значеннями якого є вже створені натуральні числа, а ключами - відповідні представлення цих чисел рядками.

Таким чином, при створенні натурального числа метод `__new__()` перевіряє

- можливість перетворення прототипу у ціле число,
- невід'ємність отриманого цілого числа та
- присутність його у словнику `__used`.

Крім того, єкземпляр цього класу не може дорівнювати цілому числу з таким самим значенням, що забезпечено перевантаженням методу `__eq__()`.

In [3]:
class nat(int):

    __used: Dict[str, Self] = {}

    def __new__(cls, prototype: Any) -> Self:
        try:
            ix = int(prototype)
        except ValueError:
            raise ValueError("invalid prototype type for nat()")
        if ix < 0:
            raise ValueError("invalid prototype value for nat()")
        # return super().__new__(cls, ix)
        sx = str(ix)
        if sx not in nat.__used:
            nat.__used[sx] = super().__new__(cls, ix)
        return nat.__used[sx]

    def __eq__(self, another: Any) -> bool:
        if type(another) != nat:
            return False
        return super().__eq__(another)

    def __ne__(self, another: Any) -> bool:
        return not (self == another)

# Рекурсивні функції

Об'єктом нашого рпроєкту є (загалом часткові) функції натуральних аргументів з натуральними значеннями.

Кожна така функція $f$ визначається кількістю своїх аргументів $n >0$ і правилом обчислення, за яким деяким (не обов'язково всім, як пояснюється нижче) кортежам аргументів $( x_{1} ,\dotsc ,x_{n})\in\mathbb{N}^{n}$ однозначно ставиться у відповідність натуральне число, яке називають значенням функції і позначаються $f(x_{1},\dotsc ,x_{n})$.

В інформатиці вважається, що відповісти на питання чи можна визначити значення $f(x_{1},\dotsc,x_{n})$ за допомогою відповідного правила не можливо без застосування цього правила до конкретного кортежу, а сам процес застосування правила не обов'язково завершується.
<br/>
У останньому випадку вважається, що значення $f( x_{1},\dotsc,x_{n})$ не визначене, що далі позначається через $f(x_{1},\dotsc,x_{n})=\uparrow$.
<br/>
Якщо ж процес застосування правила дає значення $\displaystyle y$, тоді використовується позначення $\displaystyle f( x_{1} ,\dotsc ,x_{n}) =\downarrow y$.

Проблемою такого підходу є визначення того, що є правилами обчислення.

Для вирішення цієї проблеми у моделі, що розглядається будується клас функцій, для яких постулюються інтуітивно зрозумілі правила, а інші функції вважаються непридатними для використання у програмуванні.
<br/>
Функції побудованого класу називають рекурсивними функціями, а їх клас зазвичай позначають через $\displaystyle \mathcal{R}$.

## Базисні рекурсивні функції

Спочатку визначається множина так званих базисних функцій:

- $\operatorname{z}(x)=\downarrow 0$ для всіх $x\in\mathbb{N}$ є функцією одної змінної, яка тотожньо дорівнює нулю;
- $\operatorname{s}(x) =\downarrow x+1$ для всіх $x\in\mathbb{N}$ є також функцією одної змінної;
- $\operatorname{I}_{k}^{(n)}(x_{1},\dotsc,x_{n})=\downarrow x_{k}$ для $n >0,\ 1\leq k\leq n$ і всіх $(x_{1},\dotsc,x_{n})\in\mathbb{N}^{n}$ є сім'єю функцій, члени якої є функціями від різного числа аргументів

Зрозуміло, що правила обчислень для всіх базисних функцій у такий спосіб визначені.
Всі базисні функції вважаються рекурсивними за визначенням.

$\operatorname{I}^{(1)}_1(x)=x$ для всіх натуральних $x$.

## Правила побудови нових рекурсивних функцій

### Правило суперпозиції

Якщо є рекурсивна функція від $m$ аргументів $f$ та рекурсивні функції від $n$ аргументів $g_{1},\dotsc,g_{m}$, де $m,n >0$, тоді суперпозицією цих функцій є функція $h$ від $n$ аргументів, що визначається у наступний спосіб
\begin{align*}
h(x_{1},\dotsc,x_{n}) &=\downarrow z\text{ якщо }g_{i}(x_{1},\dotsc,x_{n})=\downarrow y_{i}\text{ для всіх }i=1,\dotsc ,m,\text{ а }f(y_{1},\dotsc,y_{m})=\downarrow z\\
h(x_{1},\dotsc,x_{n}) &=\uparrow\text{ якщо попереднє не є виконаним},
\end{align*}
вважається рекурсивною функцією.

**Зауваження.** Для такої функції $h$ іноді використовується позначення $f\ast( g_{1},\dotsc,g_{m})$.

$\operatorname{one}=\operatorname{s}*(\operatorname{z})$<br/>
$\operatorname{one\_n}=\operatorname{one}*(\operatorname{I}^{(n)}_1)$

### Правило примітивної рекурсії

Якщо є рекурсивні функції $f$ від $n$ аргументів та $g$ від $n+2$ аргументів, де $n>0$, тоді примітивною рекурсією цих функцій є функція $h$ від $n+1$ аргумента, що визначається у наступний спосіб
\begin{align*}
h(0,x_{1},\dotsc,x_{n}) &=\downarrow z\text{ якщо }f( x_{1},\dotsc,x_{n})=\downarrow z\\
h(x+1,x_{1},\dotsc,x_{n}) &=\downarrow z\text{ якщо }h(x,x_{1},\dotsc,x_{n}) =\downarrow y,\text{ а }g(x,y,x_{1},\dotsc,x_{n}) =\downarrow z\\
h(x,x_{1},\dotsc,x_{n}) &=\uparrow\text{ якщо попереднє не є виконаним},
\end{align*}
вважається рекурсивною функцією.

**Зауваження.** Для такої функції $h$ іноді використовується позначення $\rho x(f,g)$.

**Примітивно рекурсивні функції.** Функції, що утворюються з базисних шляхом послідовних застосувань правил суперпозиції і примітивної рекурсії, називаються примітивно рекурсивними функціями.
Відповідний клас функції позначається через $\mathcal{PR}$.

### Правило мінімізації

Якщо є рекурсивна функція $f$ від $n+1$ змінної, де $n >0$, тоді її мінімізацією є функція $h$ від $n$ змінних, що визначається у наступний спосіб
\begin{align*}
h(x_{1},\dotsc,x_{n}) &=\downarrow z\text{ якщо }f(x,x_{1},\dotsc,x_{n}) =\downarrow y_{x}>0\text{ для всіх }0\leq x<z,\text{ а }f(z,x_{1},\dotsc,x_{n}) =\downarrow 0\\
h(x_{1},\dotsc,x_{n}) &=\uparrow\text{ якщо попереднє не є виконаним},
\end{align*}
вважається рекурсивною функцією.

**Зауваження.** Для функції $h$ іноді використовується позначення $\mu x(f(x,x_{1},\dotsc,x_{n})=0)$.

**Рекурсивні функції.** Функції, що утворюються з базисних шляхом послідовних застосувань правил суперпозиції, примітивної рекурсії та мінімізації, називаються рекурсивними функціями.

## Програмна реалізація

In [4]:
class RecFun(tuple):

    def __new__(cls, *args) -> Self:
        if not args:
            raise ValueError("no argument for RecFun()")
        if args[0] == 0:  # zero function
            if len(args) != 1:
                raise ValueError("invalid argument(s) for RecFun()")
            return super().__new__(cls, (1, lambda u: 0))
        if args[0] == 1:  # succ function
            if len(args) != 1:
                raise ValueError("invalid argument(s) for RecFun()")
            return super().__new__(cls, (1, lambda u: u + 1))
        if args[0] == 2:  # proj function
            if len(args) != 3:
                raise ValueError("invalid argument(s) for RecFun()")
            if not all(isinstance(arg, int) for arg in args[1 :]):
                raise ValueError("invalid argument(s) for RecFun()")
            if not 1 <= args[2] <= args[1]:
                raise ValueError("invalid argument(s) for RecFun()")
            return super().__new__(cls, (args[1], lambda *u: u[args[2] - 1]))
        if args[0] == 3:  # superposition of functions
            if len(args) < 3:
                raise ValueError("invalid argument(s) for RecFun()")
            f = args[1]
            gs = args[2 :]
            if not all(isinstance(arg, RecFun) for arg in args[1 :]):
                raise ValueError("invalid argument(s) for RecFun()")
            f_nargs = args[1][0]
            if f[0] != len(gs):
                raise ValueError("invalid argument(s) for RecFun()")
            if len(args) == 3:
                return super().__new__(cls, (
                    gs[0][0], lambda *u: f[1](gs[0][1](*u))))
            if not reduce(lambda u, v: u[0] == v[0], args[2 :]):
                raise ValueError("invalid argument(s) for RecFun()")
            nargs = gs[0][0]
            return super().__new__(cls, (
                args[2][0],
                lambda *u: f[1](*tuple(arg[1](u) for arg in gs))))
        if args[0] == 4:  # primitive recursion of functions
            if len(args) != 3:
                raise ValueError("invalid argument(s) for RecFun()")
            if args[1][0] + 2 != args[2][0]:
                raise ValueError("invalid argument(s) for RecFun()")
            nargs = args[1][0] + 1
            def inner(*u):
                n, x, u = 0, u[0], u[1 :]
                z = args[1][1](*u)
                print(f"n = {n}, x = {x}, u = {u}, z = {z}")
                while n < x:
                    z = args[2][1](n, z, u)
                    n += 1
                return z
            return super().__new__(cls, (nargs, inner))
        if args[0] == 5:  # minimization of a function
            if len(args) != 2:
                raise ValueError("invalid argument(s) for RecFun()")
            if args[1][0] < 2:
                raise ValueError("invalid argument(s) for RecFun()")
            nargs = args[1][0] - 1
            def inner(*u):
                x, u = 0, u[1 :]
                z = args[1][1](x, u)
                while z > 0:
                    x += 1
                    z = args[1][1](x, u)
                return z
            return super().__new__(cls, (nargs, inner))
        raise ValueError("invalid argument(s) for RecFun()")

    def __call__(self, *args: Tuple[nat]) -> nat:
        if self[0] != len(args):
            raise TypeError("invalid number of function arguments")
        if not all(isinstance(arg, nat) for arg in args):
            raise TypeError("invalid type of function argument(s)")
        return self[1](*args)

In [ ]:
id = RecFun(2, 1, 1)
succ = RecFun(1)
pr32 = RecFun(2, 3, 2)
sadd = RecFun(3, succ, pr32)
add = RecFun(4, id, sadd)
print(f"id(2) = {id(nat(2))}")
print(f"succ(5) = {succ(nat(5))}")
print(f"pr3_2(1, 2, 3) = {pr32(nat(1), nat(2), nat(3))}")
print(f"sadd(1, 5, 3) = {sadd(nat(1), nat(5), nat(3))}")
print(f"add(4, 5) = {add(nat(4), nat(5))}")

id(2) = 2
succ(5) = 6
pr3_2(1, 2, 3) = 2
sadd(1, 5, 3) = 6
n = 0, x = 4, u = (5,), z = 5
add(4, 5) = 9
